RoBERTa training from skretch made easy

In [1]:
from seaqube.nlp.roberta.seaberta import SeaBERTa
from seaqube.nlp.types import SeaQuBeWordEmbeddingsModel, SeaQuBeNLPModel2WV

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
from seaqube.tools.io import load_json, save_json
from os.path import join

In [4]:
# Import some seaqube tools:
from seaqube.nlp.tools import word_count_list
from seaqube.nlp.types import RawModelTinCan
from seaqube.nlp.seaqube_model import SeaQuBeNLPLoader, SeaQuBeCompressLoader
from seaqube.nlp.tools import tokenize_corpus

In [5]:
yoda_cites = [["fear", "is", "the", "path", "to", "the", "dark", "side", ".", "fear", "leads", "to", "anger", ".", "anger", "leads", "to", "hate", ".", "hate", "leads", "to", "suffering", "."], ["once", "you", "start", "down", "the", "dark", "path", ",", "forever", "will", "it", "dominate", "your", "destiny", ".", "consume", "you", ",", "it", "will", "."], ["always", "pass", "on", "what", "you", "have", "learned", "."], ["patience", "you", "must", "have", "my", "young", "padawan", "."], ["in", "a", "dark", "place", "we", "find", "ourselves", ",", "and", "a", "little", "more", "knowledge", "lights", "our", "way", "."], ["death", "is", "a", "natural", "part", "of", "life", ".", "rejoice", "for", "those", "around", "you", "who", "transform", "into", "the", "force", ".", "mourn", "them", "do", "not", ".", "miss", "them", "do", "not", ".", "attachment", "leads", "to", "jealously", ".", "the", "shadow", "of", "greed", ",", "that", "is", "."], ["powerful", "you", "have", "become", ",", "the", "dark", "side", "i", "sense", "in", "you", "."], ["train", "yourself", "to", "let", "go", "of", "everything", "you", "fear", "to", "lose", "."], ["feel", "the", "force", "!"], ["truly", "wonderful", "the", "mind", "of", "a", "child", "is", "."], ["do", "or", "do", "not", ".", "there", "is", "no", "try", "."], ["great", "warrior", ".", "wars", "not", "make", "one", "great", "."], ["size", "matters", "not", ".", "look", "at", "me", ".", "judge", "me", "by", "my", "size", ",", "do", "you", "?", "hmm", "?", "hmm", ".", "and", "well", "you", "should", "not", ".", "for", "my", "ally", "is", "the", "force", ",", "and", "a", "powerful", "ally", "it", "is", ".", "life", "creates", "it", ",", "makes", "it", "grow", ".", "its", "energy", "surrounds", "us", "and", "binds", "us", ".", "luminous", "beings", "are", "we", ",", "not", "this", "crude", "matter", ".", "you", "must", "feel", "the", "force", "around", "you", ";", "here", ",", "between", "you", ",", "me", ",", "the", "tree", ",", "the", "rock", ",", "everywhere", ",", "yes", ".", "even", "between", "the", "land", "and", "the", "ship", "."], ["the", "dark", "side", "clouds", "everything", ".", "impossible", "to", "see", "the", "light", ",", "the", "future", "is", "."], ["you", "will", "find", "only", "what", "you", "bring", "in", "."], ["to", "be", "jedi", "is", "to", "face", "the", "truth", ",", "and", "choose", ".", "give", "off", "light", ",", "or", "darkness", ",", "padawan", ".", "be", "a", "candle", ",", "or", "the", "night", "."], ["control", ",", "control", ",", "you", "must", "learn", "control", "!"], ["on", "many", "long", "journeys", "have", "i", "gone", ".", "and", "waited", ",", "too", ",", "for", "others", "to", "return", "from", "journeys", "of", "their", "own", ".", "some", "return", ";", "some", "are", "broken", ";", "some", "come", "back", "so", "different", "only", "their", "names", "remain", "."], ["in", "the", "end", ",", "cowards", "are", "those", "who", "follow", "the", "dark", "side", "."], ["difficult", "to", "see", ".", "always", "in", "motion", "is", "the", "future", "."], ["ready", "are", "you", "?", "what", "know", "you", "of", "ready", "?", "for", "eight", "hundred", "years", "have", "i", "trained", "jedi", ".", "my", "own", "counsel", "will", "i", "keep", "on", "who", "is", "to", "be", "trained", ".", "a", "jedi", "must", "have", "the", "deepest", "commitment", ",", "the", "most", "serious", "mind", ".", "this", "one", "a", "long", "time", "have", "i", "watched", ".", "all", "his", "life", "has", "he", "looked", "away\u2026", "to", "the", "future", ",", "to", "the", "horizon", ".", "never", "his", "mind", "on", "where", "he", "was", ".", "hmm", "?", "what", "he", "was", "doing", ".", "hmph", ".", "adventure", ".", "heh", ".", "excitement", ".", "heh", ".", "a", "jedi", "craves", "not", "these", "things", ".", "you", "are", "reckless", "."], ["secret", ",", "shall", "i", "tell", "you", "?", "grand", "master", "of", "jedi", "order", "am", "i", ".", "won", "this", "job", "in", "a", "raffle", "i", "did", ",", "think", "you", "?", "\u2018", "how", "did", "you", "know", ",", "how", "did", "you", "know", ",", "master", "yoda", "?", "\u2019", "master", "yoda", "knows", "these", "things", ".", "his", "job", "it", "is", "."], ["to", "answer", "power", "with", "power", ",", "the", "jedi", "way", "this", "is", "not", ".", "in", "this", "war", ",", "a", "danger", "there", "is", ",", "of", "losing", "who", "we", "are", "."], ["many", "of", "the", "truths", "that", "we", "cling", "to", "depend", "on", "our", "point", "of", "view", "."], ["named", "must", "your", "fear", "be", "before", "banish", "it", "you", "can", "."], ["you", "think", "yoda", "stops", "teaching", ",", "just", "because", "his", "student", "does", "not", "want", "to", "hear", "?", "a", "teacher", "yoda", "is", ".", "yoda", "teaches", "like", "drunkards", "drink", ",", "like", "killers", "kill", "."], ["do", "not", "assume", "anything", "obi-wan", ".", "clear", "your", "mind", "must", "be", "if", "you", "are", "to", "discover", "the", "real", "villains", "behind", "this", "plot", "."], ["you", "will", "know", "(", "the", "good", "from", "the", "bad", ")", "when", "you", "are", "calm", ",", "at", "peace", ".", "passive", ".", "a", "jedi", "uses", "the", "force", "for", "knowledge", "and", "defense", ",", "never", "for", "attack", "."], ["soon", "will", "i", "rest", ",", "yes", ",", "forever", "sleep", ".", "earned", "it", "i", "have", ".", "twilight", "is", "upon", "me", ",", "soon", "night", "must", "fall", "."], ["when", "you", "look", "at", "the", "dark", "side", ",", "careful", "you", "must", "be", ".", "for", "the", "dark", "side", "looks", "back", "."], ["you", "will", "know", "(", "the", "good", "from", "the", "bad", ")", "when", "you", "are", "calm", ",", "at", "peace", ".", "passive", ".", "a", "jedi", "uses", "the", "force", "for", "knowledge", "and", "defense", ",", "never", "for", "attack", "."], ["smaller", "in", "number", "are", "we", ",", "but", "larger", "in", "mind", "."], ["your", "path", "you", "must", "decide", "."], ["always", "two", "there", "are", ",", "no", "more", ",", "no", "less", ".", "a", "master", "and", "an", "apprentice", "."], ["no", "longer", "certain", ",", "that", "one", "ever", "does", "win", "a", "war", ",", "i", "am", ".", "for", "in", "fighting", "the", "battles", ",", "the", "bloodshed", ",", "already", "lost", "we", "have", ".", "yet", ",", "open", "to", "us", "a", "path", "remains", ".", "that", "unknown", "to", "the", "sith", "is", ".", "through", "this", "path", ",", "victory", "we", "may", "yet", "find", ".", "not", "victory", "in", "the", "clone", "wars", ",", "but", "victory", "for", "all", "time", "."], ["if", "no", "mistake", "you", "have", "made", ",", "losing", "you", "are", ".", "a", "different", "game", "you", "should", "play", "."], ["[", "luke", "skywalker", ":", "]", "i", "can", "\u2019", "t", "believe", "it", ".", "[", "yoda", ":", "]", "that", "is", "why", "you", "fail", "."], ["happens", "to", "every", "guy", "sometimes", "this", "does"], ["adventure", ".", "excitement", ".", "a", "jedi", "craves", "not", "these", "things", "."], ["only", "the", "dark", "lord", "of", "the", "sith", "knows", "of", "our", "weakness", ".", "if", "informed", "the", "senate", "is", ",", "multiply", "our", "adversaries", "will", "."]]*10

For roberta a location is needed where checkpoints and pre processing is saved


In [6]:
example_path = !pwd

In [7]:
main_path = join(example_path[0], "seaberta", "main")
train_path = join(example_path[0], "seaberta", "training")

Roberta needs some parameters. At least use 20 epochs for this dataset.

In [8]:
train_params = {
        "per_gpu_eval_batch_size": 4,
        "do_eval": True,
        "evaluate_during_training": False,
        "line_by_line": False,
        "should_continue": False,
        "model_name_or_path": False,
        "mlm": True,
        "do_train": True,
        "overwrite_output_dir": True,
        "overwrite_cache": False,
        "block_size": 512,
        "eval_all_checkpoints": 2,
        "server_ip": "",
        "mlm_probability": 0.15,
        "local_rank": -1,  # NO GPU,
        "no_cuda": False,
        "fp16": False,
        "fp16_opt_level": 'O1',
        "max_steps": 10,
        "warmup_steps": 10,
        "learning_rate": 5e-5,
        "per_gpu_train_batch_size": 4,
        "gradient_accumulation_steps": 4,
        "weight_decay": 0.01,
        "adam_epsilon": 1e-6,
        "max_grad_norm": 100.0,
        "save_total_limit": 10,
        "save_steps": 10,
        "logging_steps": 2,
        "seed": 0,
    }

In [9]:
roberta = SeaBERTa(main_path, train_params)

In [10]:
roberta.train(yoda_cites)

/Users/allankarlson/.conda/envs/master-thesis/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
INFO:seaqube.nlp.roberta.roberta_training:Training new model from scratch


/Users/allankarlson/Documents/01_UNI/CAPGEMINI/Capgemini_MASTER-THESIS/master-thesis/code/SeaQuBeRepo/examples/seaberta/main/main.txt 335


/Users/allankarlson/.conda/envs/master-thesis/lib/python3.7/site-packages/transformers/modeling_auto.py:797: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
INFO:seaqube.nlp.roberta.roberta_training:Training/evaluation parameters {'per_gpu_eval_batch_size': 4, 'do_eval': True, 'evaluate_during_training': False, 'line_by_line': False, 'should_continue': False, 'model_name_or_path': False, 'mlm': True, 'do_train': True, 'overwrite_output_dir': True, 'overwrite_cache': False, 'block_size': 512, 'eval_all_checkpoints': 2, 'server_ip': '', 'mlm_probability': 0.15, 'local_rank': -1, 'no_cuda': False, 'fp16': False, 'fp16_opt_level': 'O1', 'max_steps': 10, 'warmup_steps': 10, 'learning_rate': 5e-05, 'per_gpu_train_batch_size': 4, 'gradient_accumulation_steps

tokenizer PreTrainedTokenizer(name_or_path='/Users/allankarlson/Documents/01_UNI/CAPGEMINI/Capgemini_MASTER-THESIS/master-thesis/code/SeaQuBeRepo/examples/seaberta/main', vocab_size=335, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})



Epoch: 100%|██████████| 10/10 [02:20<00:00, 14.04s/it]
INFO:seaqube.nlp.roberta.roberta_training: global_step = 1, average loss = 14.833906650543213
INFO:seaqube.nlp.roberta.roberta_training:Saving model checkpoint to /Users/allankarlson/Documents/01_UNI/CAPGEMINI/Capgemini_MASTER-THESIS/master-thesis/code/SeaQuBeRepo/examples/seaberta/main/output
/Users/allankarlson/.conda/envs/master-thesis/lib/python3.7/site-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
INFO:seaqube.nlp.roberta.roberta_training:Evaluate the following checkpoints: ['/Users/allankarlson/Documents/01_UNI/CAPGEMINI/Capgemini_MASTER-THESIS/master-thesis/code/SeaQuBeRepo/examples/seaberta/main/output']


DEBUG {'per_gpu_eval_batch_size': 4, 'do_eval': True, 'evaluate_during_training': False, 'line_by_line': False, 'should_continue': False, 'model_name_or_path': False, 'mlm': True, 'do_train': True, 'overwrite_output_dir': True, 'overwrite_cache': False, 'block_size': 512, 'eval_all_checkpoints': 2, 'server_ip': '', 'mlm_probability': 0.15, 'local_rank': -1, 'no_cuda': False, 'fp16': False, 'fp16_opt_level': 'O1', 'max_steps': 10, 'warmup_steps': 10, 'learning_rate': 5e-05, 'per_gpu_train_batch_size': 4, 'gradient_accumulation_steps': 4, 'weight_decay': 0.01, 'adam_epsilon': 1e-06, 'max_grad_norm': 100.0, 'save_total_limit': 10, 'save_steps': 10, 'logging_steps': 2, 'seed': 0, 'output_dir': '/Users/allankarlson/Documents/01_UNI/CAPGEMINI/Capgemini_MASTER-THESIS/master-thesis/code/SeaQuBeRepo/examples/seaberta/main/output', 'model_type': 'roberta', 'config_name': '/Users/allankarlson/Documents/01_UNI/CAPGEMINI/Capgemini_MASTER-THESIS/master-thesis/code/SeaQuBeRepo/examples/seaberta/main'

INFO:seaqube.nlp.roberta.roberta_training:Loading features from cached file /Users/allankarlson/Documents/01_UNI/CAPGEMINI/Capgemini_MASTER-THESIS/master-thesis/code/SeaQuBeRepo/examples/seaberta/main/roberta_cached_lm_510_main_eval.txt
INFO:seaqube.nlp.roberta.roberta_training:***** Running evaluation  *****
INFO:seaqube.nlp.roberta.roberta_training:  Num examples = 0
INFO:seaqube.nlp.roberta.roberta_training:  Batch size = 4
Evaluating: 0it [00:00, ?it/s]
INFO:seaqube.nlp.roberta.roberta_training:***** Eval results  *****
INFO:seaqube.nlp.roberta.roberta_training:  perplexity = tensor(1.)


In [12]:
roberta.load_trained_model()

In [14]:
class SeaQuBeWordEmbeddingsModelSeaBERTa(SeaQuBeWordEmbeddingsModel):
    def __init__(self, seaberta: SeaBERTa):
        self.seaberta = seaberta

    def vocabs(self):
        return self.seaberta.wv.vocabs

    @property
    def wv(self):
        return self.seaberta.wv

    def word_vector(self, word):
        return self.seaberta.wv[word]

    def matrix(self):
        return self.seaberta.wv.matrix

    def context_embedding(self, words, position):
        return self.seaberta.context_embedding(words, position)

In [15]:
seaberta = SeaQuBeWordEmbeddingsModelSeaBERTa(roberta)

In [17]:
roberta.context_embedding(["t"], 1)

100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


array([[-0.29637682,  0.17495562, -0.32151467,  0.28507614,  0.36918864,
        -0.68209875, -0.5761418 ,  0.0354082 ,  0.46975166, -0.51650286,
         0.09801871,  0.10970846,  0.56172836,  0.58035207,  0.5220239 ,
         0.23842059,  0.2670231 ,  0.3038025 ,  0.23341294, -0.90406716,
         0.39077038, -0.43437737,  0.43068188, -0.22495733, -0.7062916 ,
        -0.51804125, -0.01821473, -0.97982574, -0.44811812, -0.06091636,
        -0.71079534, -0.43467906,  0.14559722, -0.09732482,  0.69507766,
         0.12236589,  0.92346823,  1.1685666 ,  0.04599249,  0.36656547,
         0.41244188, -0.5553379 ,  0.49144047,  0.19862236, -0.8497895 ,
        -0.92402613,  0.42484593,  0.5010946 , -0.03935878,  0.8109755 ,
         0.14996868,  0.01971561,  1.3662062 ,  0.84346354, -1.0190625 ,
        -0.42705184, -0.41649067,  0.5931656 ,  0.2082949 ,  0.06140208,
         0.35606593, -0.52583665,  0.09678748,  1.4051244 ,  0.69136775,
         0.00541956,  0.34543437,  0.21721885, -0.4

In [18]:
roberta.wv.vocabs

['',
 't',
 '.',
 'th',
 'a',
 'i',
 ',',
 'y',
 'the',
 'w',
 'you',
 'm',
 'd',
 'f',
 's',
 'o',
 'h',
 'l',
 'b',
 'c',
 'to',
 'p',
 'is',
 'no',
 'for',
 'j',
 'be',
 'k',
 'in',
 'wh',
 'of',
 'not',
 'g',
 'are',
 'do',
 'on',
 'ha',
 're',
 'and',
 'it',
 'dar',
 'know',
 'have',
 'dark']

In [19]:
tin_can = RawModelTinCan(seaberta, word_count_list(yoda_cites))
nlp = SeaQuBeNLPLoader.load_model_from_tin_can(tin_can, "seaberta")

100% (1 of 1) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [20]:
nlp("Yoda and more")

100% (3 of 3) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


IndexError: list index out of range